In [6]:
import os
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableSequence
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import Qdrant
from langchain.embeddings import FastEmbedEmbeddings
from langchain.chains import LLMChain
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys and URLs from environment variables
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

# Define the prompt template with an additional instruction
template = """You are a medical AI assistant specialized in answering questions about urinary tract infection in children.
You will be provided with excerpts from a long document and a related question.
Respond in a conversational manner and in the same language as the question also you can translate to question language.
If you don't know the answer, simply say, "Hmm, I'm not sure."
Do not attempt to make up an answer.
Question: {input}
=========
{context}
=========
Answer in Markdown:"""

# Create a PromptTemplate object with the defined template
prompt = PromptTemplate(template=template, input_variables=["input", "context"])

# Initialize the language model with specific parameters
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

# Create the document chain using the language model and the map-reduce strategy
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

# Create embeddings using FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings()

# Set up the Qdrant vector store to connect to the existing collection
vectorstore = Qdrant.from_existing_collection(
    embedding=embeddings,         # Embedding model to use
    url=qdrant_url,               # URL for the Qdrant instance
    collection_name="uti",        # Name of the existing Qdrant collection
    api_key=qdrant_api_key,       # API key for authentication
    path=None                     # Path to the existing collection, set to None if not needed
)

# Create a retriever object from the vector store for querying
retriever = vectorstore.as_retriever(
    search_type="similarity",     # Type of search to perform, here it's similarity search
    search_kwargs={"k": 3}        # Additional search parameters, 'k' specifies the number of top results to return
)

# Create the history-aware retriever for context-aware querying
history_aware_retriever = create_history_aware_retriever(
    llm=RunnableSequence(prompt | llm),  # Language model sequence combining prompt and LLM
    retriever=retriever,                 # Retriever for fetching relevant documents
    prompt=prompt                        # Prompt template for the retriever
)

# Create the LLM chain for generating answers
qa_chain = LLMChain(
    llm=llm,                # Language model to use for generating answers
    prompt=prompt           # Prompt template to guide the answer generation
)

# Argument Explanations:
# - template: The text template that defines how the AI should respond, with placeholders for input and context.
# - input_variables: Variables that will be replaced in the template (e.g., "input" and "context").
# - temperature: Parameter for the language model that controls the randomness of the output (0 means deterministic).
# - model_name: Specifies the model version of the language model.
# - embedding: The embedding model used to convert text into vector representations.
# - url: The URL of the Qdrant instance where the documents are stored.
# - collection_name: The name of the existing collection in Qdrant.
# - api_key: The API key for authenticating with the Qdrant instance.
# - search_type: The type of search to perform, in this case, a similarity search to find similar documents.
# - search_kwargs: Additional keyword arguments for the search, 'k' specifies the number of similar results to return.
# - llm: The language model used for generating responses and integrated into various chains.
# - retriever: The component used to fetch relevant documents from the vector store based on the query.
# - prompt: The prompt template guiding the AI on how to respond to questions.
# - RunnableSequence: Combines the prompt and the language model to create a sequence for the history-aware retriever.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
question = "What imaging tests are recommended for babies younger than 6 months with atypical UTI?"
context = "The NICE guidelines for the diagnosis and management of urinary tract infections (UTIs) in children under 16 provide comprehensive recommendations. The guidelines cover various aspects such as symptoms and signs, diagnostic procedures, acute management, prevention of recurrence, and follow-up care. They emphasize the importance of accurate diagnosis to avoid unnecessary treatments and to manage UTIs effectively. The guidelines also include specific recommendations for urine testing, imaging tests, and the use of antibiotics. Imaging tests, in particular, are recommended based on the age of the child, the type of UTI (acute upper or lower), and the response to initial treatment. For instance, certain imaging tests are suggested for babies under 6 months with atypical UTI to identify structural abnormalities and ensure prompt and appropriate management."
result = qa_chain.invoke({"input": question, "context": context, "chat_history": []})
print(result)

{'input': 'What imaging tests are recommended for babies younger than 6 months with atypical UTI?', 'context': 'The NICE guidelines for the diagnosis and management of urinary tract infections (UTIs) in children under 16 provide comprehensive recommendations. The guidelines cover various aspects such as symptoms and signs, diagnostic procedures, acute management, prevention of recurrence, and follow-up care. They emphasize the importance of accurate diagnosis to avoid unnecessary treatments and to manage UTIs effectively. The guidelines also include specific recommendations for urine testing, imaging tests, and the use of antibiotics. Imaging tests, in particular, are recommended based on the age of the child, the type of UTI (acute upper or lower), and the response to initial treatment. For instance, certain imaging tests are suggested for babies under 6 months with atypical UTI to identify structural abnormalities and ensure prompt and appropriate management.', 'chat_history': [], 't

In [9]:
question = "What is the recommended management for babies and children over 3 months with an acute upper UTI?"
context = "The NICE guidelines for the diagnosis and management of urinary tract infections (UTIs) in children under 16 provide comprehensive recommendations. The guidelines cover various aspects such as symptoms and signs, diagnostic procedures, acute management, prevention of recurrence, and follow-up care. They emphasize the importance of accurate diagnosis to avoid unnecessary treatments and to manage UTIs effectively. The guidelines also include specific recommendations for urine testing, imaging tests, and the use of antibiotics. Imaging tests, in particular, are recommended based on the age of the child, the type of UTI (acute upper or lower), and the response to initial treatment. For instance, certain imaging tests are suggested for babies under 6 months with atypical UTI to identify structural abnormalities and ensure prompt and appropriate management."
result = qa_chain.invoke({"input": question, "context": context, "chat_history": []})
print(result)

{'input': 'What is the recommended management for babies and children over 3 months with an acute upper UTI?', 'context': 'The NICE guidelines for the diagnosis and management of urinary tract infections (UTIs) in children under 16 provide comprehensive recommendations. The guidelines cover various aspects such as symptoms and signs, diagnostic procedures, acute management, prevention of recurrence, and follow-up care. They emphasize the importance of accurate diagnosis to avoid unnecessary treatments and to manage UTIs effectively. The guidelines also include specific recommendations for urine testing, imaging tests, and the use of antibiotics. Imaging tests, in particular, are recommended based on the age of the child, the type of UTI (acute upper or lower), and the response to initial treatment. For instance, certain imaging tests are suggested for babies under 6 months with atypical UTI to identify structural abnormalities and ensure prompt and appropriate management.', 'chat_histo

{'input': 'What is the recommended management for babies and children over 3 months with an acute upper UTI?', 'context': 'The NICE guidelines for the diagnosis and management of urinary tract infections (UTIs) in children under 16 provide comprehensive recommendations. The guidelines cover various aspects such as symptoms and signs, diagnostic procedures, acute management, prevention of recurrence, and follow-up care. They emphasize the importance of accurate diagnosis to avoid unnecessary treatments and to manage UTIs effectively. The guidelines also include specific recommendations for urine testing, imaging tests, and the use of antibiotics. Imaging tests, in particular, are recommended based on the age of the child, the type of UTI (acute upper or lower), and the response to initial treatment. For instance, certain imaging tests are suggested for babies under 6 months with atypical UTI to identify structural abnormalities and ensure prompt and appropriate management.', 'chat_history': [], 'text': "**According to the NICE guidelines, for babies and children over 3 months with an acute upper UTI, the recommended management is...**\n\n* **Antibiotic treatment**: The guidelines recommend treating acute upper UTI with antibiotics, with a duration of treatment ranging from 7 to 14 days, depending on the severity of the infection and the child's response to treatment.\n* **Imaging tests**: Imaging tests, such as ultrasound or MRI, may be recommended to rule out any underlying structural abnormalities, especially in children with atypical symptoms or those who do not respond to initial treatment.\n\nPlease note that these recommendations are based on the NICE guidelines and may vary depending on individual circumstances. It's always best to consult with a healthcare professional for personalized advice."}